# Euclidean Traveling Salesman Problem Speed Up

In [1]:
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import random
from numba import jit

In [ ]:
@jit(nopython=True)
def swapMutate(population,  n):
    """
    функция высчитывания мутаций у популяции:
    меняет два рандомных элемента у каких-то 
    представителей популяции
    population - соответственно популяция
    n - количество мутаций в популяции
    """
    indices = np.random.randint(low=0, high=population.shape[0], size=n)
    mutations = np.copy(population[indices])
    for i in range(n):
        index1, index2 = np.random.randint(low=0, high=population.shape[1], size=2)
        mutations[i, index1], mutations[i, index2] = \
        mutations[i, index2], mutations[i, index1]
    return mutations   

@jit(nopython=True)
def invMutate(population, n):
    """
    Inversion Mutation
    """
    indices = np.random.randint(low=0, high=population.shape[0], size=n)
    mutations = np.copy(population[indices])
    for i in range(n):
        index1, index2 = np.random.randint(low=1, high=population.shape[1], size=2)
        mutations[i, index1: index2+1] = mutations[i, index2: index1-1:-1]
    return mutations  

### Numba

In [ ]:
@jit(nopython=True)
def PMX(specimen1, specimen2):
    """
    Partitially-Mapped Crossover
    """
    index1, index2 = np.sort(np.random.randint(low=0, high=len(specimen1), size=2))
    offspring = np.ones(len(specimen1))*-1
    offspring[index1 : index2 + 1] = specimen1[index1 : index2 + 1]
    k = 0

    for el in specimen2:
        if k == index1:
            k = index2 + 1
        
        set_ = specimen1[index1 : index2 + 1]
        len_ = len(np.where(set_==el)[0])
        
        if len_==0:
            offspring[k] = el
            k += 1

    return offspring

@jit(nopython=True)
def CX(specimen1, specimen2):
    """
    Cycle Crossover
    """
    argparent1 = np.argsort(specimen1)
    argparent2 = np.argsort(specimen2) #индексы отсорт. массива 2
    
    offspring = np.ones(len(specimen1))*-1
    offspring = offspring.astype(int)
    offspring[0] = specimen1[0]
    ind = 0
    curr_argp = argparent2
    curr_spec = specimen1
    for i in range(len(offspring)-1):
        ind = curr_argp[offspring[ind]]
        if (offspring[ind] != -1):
            ind  = np.argmin(offspring)
            if (np.all(curr_spec ==specimen1)):
                curr_argp = argparent1
                curr_spec = specimen2
            else:
                curr_argp = argparent2
                curr_spec = specimen1
        offspring[ind] = curr_spec[ind]        
    return offspring

@jit(nopython=True)
#само скрещивание популяции   
def getCrossover(population, func):
    """
    скрещивание популяции - смена поколения
    population - популяция
    func - функция скрещивания
    """
    m = population.shape[0] * (population.shape[0]+1) / 2
    generation = np.zeros((int(m), population.shape[1]))
    k = 0
    for i in range(population.shape[0]):
        for j in range(i, population.shape[0]):
            offspring = func(population[i], population[j])
            generation[k] = offspring
            k += 1
    return generation

In [4]:
@jit(nopython=True)
def createPopulation(Graph, n):
    """
    создание популяции путем
    рандомного перемешивания 
    
    n - размер популяции
    """
    specimen = np.arange(Graph.shape[0]) 
    population = np.zeros((n, Graph.shape[0]))
    for i in range(n):
        arr = np.copy(specimen)
        np.random.shuffle(arr)
        population[i] = arr
    return population

@jit(nopython=True)
#длина пути для какого-то представителя популяции
def pathLength(Graph, specimen):
    """
    считает длину цикла, соответствующего 
    массиву индексов specimen
    последнее, замыкающее ребро цикла -
    считается между последним и 1ым эл-ами массива
    
    specimen - массив индексов вершин графа - особь популяции
    """
    length = 0
    for i in range(Graph.shape[0]):
        length += np.linalg.norm(Graph[int(specimen[i])] - \
            Graph[int(specimen[(i + 1) % Graph.shape[0]])])
    return length

@jit(nopython=True)
def sortPopulation(Graph, population):
    """
    функция отбора - сортировка популяции  по длине цикла
    """
    lengths = [pathLength(Graph, population[i]) for i in range(len(population))]
    lengths = np.array(lengths)
    
    indexes = lengths.argsort()
    population = population[indexes]
    #population = np.array(sorted(population, key= lambda x:pathLength(Graph, x)))
    return population
    
@jit(nopython=True)
#сам генетический алгоритм    
def Genetic(Graph, iter_count, popul_count, mutate_count=-1,
           mutate_func=swapMutate, cross_func=PMX):
    """
    Генетический алогритм
    Graph - граф в виде массива точек на плоскости
    iter_count - количество итераций
    popul_count - размер популяции
    mutate_count - количество мутаций
    mutate_func - функция мутации
    cross_func - функция скрещивания
    выводит массив индексов - "лучшую" особь популяции
    """
    population = createPopulation(Graph, popul_count)
    
    if (mutate_count == -1):
        mutate_count = int(popul_count / 4)
    
    for i in range(iter_count):
        mutations = mutate_func(population, mutate_count)
        offsprings = getCrossover(population, cross_func)

        population = np.vstack((population, mutations, 
                                offsprings))
        population = sortPopulation(Graph, population)[:popul_count]

    return population[0]

@jit(nopython=True)
def getRandomGraph(N):
    """
    генерирует произвольный граф размера N
    каждая координата генерируется из равномерного
    распеделения на отрезке [0, N]
    """
    Graph = np.zeros((N, 2))

    for i in range(N):
        Graph[i] = [np.random.uniform(0, N), np.random.uniform(0, N)]
    return Graph

@jit(nopython=True)
def getCycleIndices(indices):
    """
    возвращает "зацикленный" массив индексов -
    для удобства вывода на график
    """
    new_indices = np.ones(len(indices) + 1)
    new_indices[:len(indices)] = indices
    new_indices[-1] = indices[0]
    return new_indices

In [6]:
graph_sizes = map(int, np.linspace(10, 400, 20))
iter_count = 100 # количество итераций
popul_count = 20 #  размеры популяций
times_jit = np.zeros(20)

for k, graph_size in enumerate(graph_sizes):
    Graph = getRandomGraph(graph_size)
    work_time = %timeit -o getCycleIndices(Genetic(Graph, iter_count, popul_count))
    times_jit[k] = work_time.average

170 ms ± 596 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
468 ms ± 7.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
793 ms ± 6.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.12 s ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.46 s ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.05 s ± 216 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.38 s ± 460 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.93 s ± 470 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.54 s ± 314 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.39 s ± 241 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.64 s ± 47.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.56 s ± 598 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.43 s ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.38 s ± 793 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.53 s ± 1.16 s per loop (mean ± std. dev. of 7 runs, 1 lo

### Pure python and numpy

In [7]:
def swapMutate(population,  n):
    """
    функция высчитывания мутаций у популяции:
    меняет два рандомных элемента у каких-то 
    представителей популяции
    population - соответственно популяция
    n - количество мутаций в популяции
    """
    indices = np.random.randint(low=0, high=population.shape[0], size=n)
    mutations = np.copy(population[indices])
    for i in range(n):
        index1, index2 = np.random.randint(low=0, high=population.shape[1], size=2)
        mutations[i, index1], mutations[i, index2] = \
        mutations[i, index2], mutations[i, index1]
    return mutations   

def invMutate(population, n):
    """
    Inversion Mutation
    """
    indices = np.random.randint(low=0, high=population.shape[0], size=n)
    mutations = np.copy(population[indices])
    for i in range(n):
        index1, index2 = np.random.randint(low=1, high=population.shape[1], size=2)
        mutations[i, index1: index2+1] = mutations[i, index2: index1-1:-1]
    return mutations

def PMX(specimen1, specimen2):
    """
    Partitially-Mapped Crossover
    """
    index1, index2 = np.sort(np.random.randint(low=0, high=len(specimen1), size=2))
    offspring = np.ones(len(specimen1))*-1
    offspring[index1 : index2 + 1] = specimen1[index1 : index2 + 1]
    k = 0

    for el in specimen2:
        if k == index1:
            k = index2 + 1
        
        set_ = specimen1[index1 : index2 + 1]
        len_ = len(np.where(set_==el)[0])
        
        if len_==0:
            offspring[k] = el
            k += 1

    return offspring

def CX(specimen1, specimen2):
    """
    Cycle Crossover
    """
    argparent1 = np.argsort(specimen1)
    argparent2 = np.argsort(specimen2) #индексы отсорт. массива 2
    
    offspring = np.ones(len(specimen1))*-1
    offspring = offspring.astype(int)
    offspring[0] = specimen1[0]
    ind = 0
    curr_argp = argparent2
    curr_spec = specimen1
    for i in range(len(offspring)-1):
        ind = curr_argp[offspring[ind]]
        if (offspring[ind] != -1):
            ind  = np.argmin(offspring)
            if (np.all(curr_spec ==specimen1)):
                curr_argp = argparent1
                curr_spec = specimen2
            else:
                curr_argp = argparent2
                curr_spec = specimen1
        offspring[ind] = curr_spec[ind]        
    return offspring

#само скрещивание популяции   
def getCrossover(population, func):
    """
    скрещивание популяции - смена поколения
    population - популяция
    func - функция скрещивания
    """
    m = population.shape[0] * (population.shape[0]+1) / 2
    generation = np.zeros((int(m), population.shape[1]))
    k = 0
    for i in range(population.shape[0]):
        for j in range(i, population.shape[0]):
            offspring = func(population[i], population[j])
            generation[k] = offspring
            k += 1
    return generation


def createPopulation(Graph, n):
    """
    создание популяции путем
    рандомного перемешивания 
    
    n - размер популяции
    """
    specimen = np.arange(Graph.shape[0]) 
    population = np.zeros((n, Graph.shape[0]))
    for i in range(n):
        arr = np.copy(specimen)
        np.random.shuffle(arr)
        population[i] = arr
    return population

#длина пути для какого-то представителя популяции
def pathLength(Graph, specimen):
    """
    считает длину цикла, соответствующего 
    массиву индексов specimen
    последнее, замыкающее ребро цикла -
    считается между последним и 1ым эл-ами массива
    
    specimen - массив индексов вершин графа - особь популяции
    """
    length = 0
    for i in range(Graph.shape[0]):
        length += np.linalg.norm(Graph[int(specimen[i])] - \
            Graph[int(specimen[(i + 1) % Graph.shape[0]])])
    return length

def sortPopulation(Graph, population):
    """
    функция отбора - сортировка популяции  по длине цикла
    """
    lengths = [pathLength(Graph, population[i]) for i in range(len(population))]
    lengths = np.array(lengths)
    
    indexes = lengths.argsort()
    population = population[indexes]
    #population = np.array(sorted(population, key= lambda x:pathLength(Graph, x)))
    return population


#сам генетический алгоритм    
def Genetic(Graph, iter_count, popul_count, mutate_count=-1,
           mutate_func=swapMutate, cross_func=PMX):
    """
    Генетический алогритм
    Graph - граф в виде массива точек на плоскости
    iter_count - количество итераций
    popul_count - размер популяции
    mutate_count - количество мутаций
    mutate_func - функция мутации
    cross_func - функция скрещивания
    выводит массив индексов - "лучшую" особь популяции
    """
    population = createPopulation(Graph, popul_count)
    
    if (mutate_count == -1):
        mutate_count = int(popul_count / 4)
    
    for i in range(iter_count):
        mutations = mutate_func(population, mutate_count)
        offsprings = getCrossover(population, cross_func)

        population = np.vstack((population, mutations, 
                                offsprings))
        population = sortPopulation(Graph, population)[:popul_count]

    return population[0]   

def getRandomGraph(N):
    """
    генерирует произвольный граф размера N
    каждая координата генерируется из равномерного
    распеделения на отрезке [0, N]
    """
    Graph = np.zeros((N, 2))

    for i in range(N):
        Graph[i] = [np.random.uniform(0, N), np.random.uniform(0, N)]
    return Graph

def getCycleIndices(indices):
    """
    возвращает "зацикленный" массив индексов -
    для удобства вывода на график
    """
    new_indices = np.ones(len(indices) + 1)
    new_indices[:len(indices)] = indices
    new_indices[-1] = indices[0]
    return new_indices

In [ ]:
graph_sizes = map(int, np.linspace(10, 400, 20))
iter_count = 100 # количество итераций
popul_count = 20 #  размеры популяций
times_ord = np.zeros(20)

for k, graph_size in enumerate(graph_sizes):
    Graph = getRandomGraph(graph_size)
    work_time = %timeit -o getCycleIndices(Genetic(Graph, iter_count, popul_count))
    times_ord[k] = work_time.average

4.52 s ± 593 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
11.7 s ± 675 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
19 s ± 1.85 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
29.4 s ± 4.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
37.8 s ± 5.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
50.3 s ± 1.37 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
57.3 s ± 758 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 8s ± 3.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 16s ± 2.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 26s ± 1.94 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 35s ± 2.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 40s ± 2.37 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 49s ± 1.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 57s ± 6.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
np.save('times_ord.npy', times_ord)
np.save('times_jit.npy', times_jit)